## Column Transformer
### Problem w/o CT
- every column that require preprocessing will be converted into numpy array and then it need to be converted into df again

## Without CT

In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv(r"C:\Users\Admin\Documents\Jupyter Notebook\Machine Learning (CampusX)\Dataset\covid_toy.csv")

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [11]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [12]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(['has_covid'],axis=1),df['has_covid'],test_size=0.2,
                                                 random_state=2)

In [13]:
X_train.shape,X_test.shape

((80, 5), (20, 5))

## Columns
1. gender - ohe
2. fever - fill values
3. cough - ordinal
4. city - ohe
5. has_covid= label

## 1. Aam zindagi

In [15]:
# Fever - adding simple imputer to fever column
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])

# also x test
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [16]:
## Cough <-- Ordinal
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

In [21]:
## Gender
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

After encoding, you won’t have the original Gender column anymore.
Instead, you'll have a new column called something like Gender_Male (assuming "Female" was dropped).
The Gender_Male column will have a value of 1 if the gender is "Male" and 0 if it's "Female".
So, you’ll end up with only one new column after encoding (Gender_Male), not two. This avoids multicollinearity because we don’t have two columns representing "Male" and "Female" at the same time.



In [22]:
## Extracting age from X_train
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

## Also test
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [23]:
X_train_age

array([[82],
       [65],
       [69],
       [20],
       [34],
       [84],
       [23],
       [ 8],
       [34],
       [56],
       [69],
       [27],
       [64],
       [66],
       [38],
       [11],
       [14],
       [46],
       [38],
       [75],
       [25],
       [83],
       [47],
       [60],
       [12],
       [34],
       [42],
       [19],
       [24],
       [81],
       [64],
       [51],
       [40],
       [ 6],
       [49],
       [82],
       [42],
       [72],
       [20],
       [19],
       [75],
       [79],
       [59],
       [ 5],
       [25],
       [14],
       [19],
       [47],
       [51],
       [ 5],
       [68],
       [19],
       [54],
       [73],
       [65],
       [23],
       [26],
       [49],
       [11],
       [27],
       [65],
       [65],
       [50],
       [55],
       [12],
       [83],
       [10],
       [18],
       [16],
       [27],
       [44],
       [74],
       [20],
       [ 5],
       [24],
       [22],
       [71],

In [25]:
## Concatenating
X_train_transformed = np.concatenate((X_train_age,X_train_gender_city,X_train_fever,X_train_cough),axis=1)
# for test
X_test_transformed = np.concatenate((X_test_age,X_test_gender_city,X_test_fever,X_test_cough),axis=1)
X_train_transformed.shape

(80, 7)

In [26]:
X_train_transformed

array([[ 82.        ,   0.        ,   0.        ,   0.        ,
          0.        , 102.        ,   1.        ],
       [ 65.        ,   0.        ,   0.        ,   0.        ,
          1.        ,  98.        ,   0.        ],
       [ 69.        ,   0.        ,   0.        ,   0.        ,
          1.        ,  98.        ,   1.        ],
       [ 20.        ,   1.        ,   1.        ,   0.        ,
          0.        , 102.        ,   1.        ],
       [ 34.        ,   1.        ,   0.        ,   1.        ,
          0.        ,  98.        ,   1.        ],
       [ 84.        ,   0.        ,   0.        ,   0.        ,
          0.        , 100.79166667,   0.        ],
       [ 23.        ,   1.        ,   0.        ,   0.        ,
          1.        , 100.79166667,   0.        ],
       [  8.        ,   0.        ,   0.        ,   1.        ,
          0.        , 101.        ,   0.        ],
       [ 34.        ,   0.        ,   1.        ,   0.        ,
          0.    

## Mentos Zindagi

In [36]:
from sklearn.compose import ColumnTransformer

In [40]:
transformer = ColumnTransformer(transformers=[
    ('trf1',SimpleImputer(),['fever']),
    ('trf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('trf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [42]:
transformer.fit_transform(X_train).shape

(80, 7)

In [44]:
transformer.fit_transform(X_test).shape

(20, 7)

In [39]:
X_train

,age,gender,fever,cough,city
35,82,Female,102.0,Strong,Bangalore
11,65,Female,98.0,Mild,Mumbai
84,69,Female,98.0,Strong,Mumbai
44,20,Male,102.0,Strong,Delhi
73,34,Male,98.0,Strong,Kolkata
...,...,...,...,...,...
43,22,Female,99.0,Mild,Bangalore
22,71,Female,98.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
15,70,Male,103.0,Strong,Kolkata
